# **Machine Learning Workflow Orchestration**

Orchestration refers to the coordination and management of various tasks, resources, and processes involved in the end-to-end machine learning lifecycle. This includes:

1. Data Preparation and Management
2. Model Training
3. Experimentation and Evaluaiton
4. Model Deployment
5. Monitor and Management
6. Automation of repetitive tasks

### **Introducing Prefect**  
Prefect is an open-source orchestration and observability platform that empowers developers to build and scale resilient code quickly, turning their Python scripts into resilient, recurring workflows.

Prefect streamlines the orchestration of machine learning workflows by providing a flexible, scalable, and reliable framework for building, deploying, and managing complex data pipelines with ease. It empowers data scientists and engineers to focus on building machine learning models and solving business problems while abstracting away the complexities of workflow management and execution.

Prefect versions:
- Prefect 1.x AKA Prefect Core
- Prefect 2.x AKA Prefect Orion

### **Why Prefect?**
- Python based open source tool  
- Manage ML Pipelines  
- Schedule and Monitor the flow  
- Gives observability into failures  
- Native dask integration for scaling (Dask is used for parallel computing)


### **Creating and activating a Virtual Environment**
In order to install prefect, create a virtual environment:
> `$ python -m venv .mlops_env`  

Enter the Virtual Environment using below mentioned command:
> `$ .mlops_env\Scripts\activate`

***

### **Installing Prefect 2.x**
Now install Prefect:
> `$ pip install prefect`  

OR  if you have Prefect 1, upgrade to Prefect 2 using this command:  
> `$ pip install -U prefect`  

OR to install a specific version:  
> `$ pip install prefect==2.4`  

***

### **Check Prefect Version**
Check the prefect version:
> `$ prefect version`

***

### **Running Prefect Dashboard (UI)**

> `$ prefect server start`  

```
 ___ ___ ___ ___ ___ ___ _____
| _ \ _ \ __| __| __/ __|_   _|
|  _/   / _|| _|| _| (__  | |
|_| |_|_\___|_| |___\___| |_|

Configure Prefect to communicate with the server with:

    prefect config set PREFECT_API_URL=http://127.0.0.1:4200/api

View the API reference documentation at http://127.0.0.1:4200/docs

Check out the dashboard at http://127.0.0.1:4200
```
***

**Note - In one of the earliest update of Prefect Orion, in Windows OS, if your path contains spaces, it will generate error (as mentioned below) when you try to run prefect orion. Sharing this so that you know what it is if you see it.**

```
___ ___ ___ ___ ___ ___ _____    ___  ___ ___ ___  _  _
| _ \ _ \ __| __| __/ __|_   _|  / _ \| _ \_ _/ _ \| \| |
|  _/   / _|| _|| _| (__  | |   | (_) |   /| | (_) | .` |
|_| |_|_\___|_| |___\___| |_|    \___/|_|_\___\___/|_|\_|
Configure Prefect to communicate with the server with:
    prefect config set PREFECT_API_URL=http://127.0.0.1:4200/api
View the API reference documentation at http://127.0.0.1:4200/docs
Check out the dashboard at http://127.0.0.1:4200/
Usage: uvicorn [OPTIONS] APP

Try 'uvicorn --help' for help.

Error: Got unexpected extra argument (prefect.orion.api.server:create_app)
Orion stopped!
```

<img src="images/prefect_dashboard.JPG">

## **Refactoring the ML Workflow**

If you've written the entire workflow code cell by cell in a Jupyter Notebook without explicitly defining functions, you may encounter difficulties when trying to visualize and monitor your flows in the Prefect dashboard.

Prefect works best when workflows are organized into modular functions, with each function representing a task in your workflow. This allows Prefect to track task dependencies, visualize the workflow graph, and provide detailed execution logs and status updates in the dashboard.

However, if you've written your workflow code directly in a Jupyter Notebook without defining functions, you can still use Prefect to run your workflows, but you may miss out on some of the dashboard's features and benefits.

To address this, you can refactor your workflow code to extract each step into separate functions, and then import these functions into your notebook. This way, you can maintain the convenience of writing and experimenting with code in a notebook while also leveraging Prefect's capabilities for workflow orchestration and monitoring.

Once you've refactored your code to use functions, you can run your flows as usual using Prefect's CLI or Python API, and you'll be able to visualize and monitor them in the Prefect dashboard.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [2]:
def load_data(file_path):
    """
    Load data from a CSV file.
    """
    return pd.read_csv(file_path)

In [3]:
def split_inputs_output(data, inputs, output):
    """
    Split features and target variables.
    """
    X = data[inputs]
    y = data[output]
    return X, y

In [4]:
def split_train_test(X, y, test_size=0.25, random_state=0):
    """
    Split data into train and test sets.
    """
    return train_test_split(X, y, test_size=test_size, random_state=random_state)

In [5]:
def clean(raw_text):
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    sentence = re.sub("[^a-zA-Z]|READ MORE", " ", raw_text)
    sentence = sentence.lower()
    tokens = nltk.word_tokenize(sentence)
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    stop_words = set(stopwords.words('english'))
    cleaned_tokens = [word for word in lemmatized_tokens if word.lower() not in stop_words]
    return " ".join(cleaned_tokens)

In [6]:
def preprocess(X_train, X_test, y_train, y_test):
    """
    cleaning the text data before hand.
    """
    vect = CountVectorizer(preprocessor=clean, max_features=5000)
    X_train_bow = vect.fit_transform(X_train)
    X_test_bow = vect.transform(X_test)
    return X_train_bow, X_test_bow, y_train, y_test

In [7]:
def train_model(X_train_bow, y_train, hyperparameters):
    """
    Training the machine learning model.
    """
    model = LogisticRegression(**hyperparameters)
    model.fit(X_train_bow, y_train)
    return model

In [8]:
def evaluate_model(model, X_train_scaled, y_train, X_test_scaled, y_test):
    """
    Evaluating the model.
    """
    y_train_pred = model.predict(X_train_scaled)
    y_test_pred = model.predict(X_test_scaled)

    train_score = metrics.accuracy_score(y_train, y_train_pred)
    test_score = metrics.accuracy_score(y_test, y_test_pred)
    
    return train_score, test_score

In [9]:
def workflow():
    DATA_PATH = 'product_reviews.csv'
    INPUT = 'review_text'
    OUTPUT = 'sentiment'
    HYPERPARAMETERS = {'max_iter':1000,
                       'C':1,
                       'class_weight':'balanced',
                       'l1_ratio':0.6,
                       'penalty':'elasticnet',
                       'solver':'saga'}
    
    # Load data
    review = load_data(DATA_PATH)

    # Identify Inputs and Output
    X, y = split_inputs_output(review, INPUT, OUTPUT)

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = split_train_test(X, y)

    # Preprocess the data
    X_train_bow, X_test_bow, y_train, y_test = preprocess(X_train, X_test, y_train, y_test)

    # Build a model
    model = train_model(X_train_bow, y_train, HYPERPARAMETERS)
    
    # Evaluation
    train_score, test_score = evaluate_model(model, X_train_bow, y_train, X_test_bow, y_test)
    
    print("Train Score:", train_score)
    print("Test Score:", test_score)


In [10]:
if __name__ == "__main__":
    workflow()

Train Score: 0.9341681130671956
Test Score: 0.9070286351803645
